In [1]:
import pandas as pd
import numpy as np
import json
import csv
#1.确定这几个数据都是干什么的
swiping_card_dataframe = pd.read_csv('./bus_swiping_card_info1.csv')
card_info_dataframe = pd.read_csv('./card_info.csv')
bus_swip_dataframe = pd.read_excel('公交数据.xlsx',sheet_name='刷卡数据',header=0,engine='openpyxl')
bus_station_dataframe = pd.read_excel('公交数据.xlsx',sheet_name='站点数据',header=0,engine='openpyxl')
#swiping_card_dataframe ： 每次刷卡的信息：几号线，第几站，车辆的id，刷卡的id，刷卡的时间
#card_info_dataframe ： 每次刷卡的信息：几号线，第几站，车辆的id，刷卡的id，刷卡的时间也是上面的信息，但是时间是去年的。
#用去年的时间的同比数据，去预测今年的同比数据，看是否一致，如果一致，说明经济发展没有变化，如果变大，说明经济好转
#bus_swip_dataframe ：一天的统计信息：各条线路、各个站点的刷卡数量信息
#bus_station_dataframe ： 站点的统计信息：各个站点的经纬度

In [2]:
#从card_info_dataframe中找出所有用户一天内的od对信息
card_info_dataframe['custom_day'] = card_info_dataframe['custom_time'].apply(lambda x:x.split(' ')[0])
card_info_dataframe['custom_precise_time'] = card_info_dataframe['custom_time'].apply(lambda x:x.split(' ')[1])
#sum(card_info_dataframe['partitionday']==card_info_dataframe['custom_day'])
#card_info_dataframe
card_info_dataframe.drop(labels=['custom_time','card_type','consume','data_src','data_load_time','partitionday'],axis=1,inplace=True)
card_info_dataframe
day_0929_dataframe = card_info_dataframe[card_info_dataframe['custom_day']=='2023-09-29']
day_0929_dataframe

,card_identity,line_identity,car_identity,station_identity,custom_day,custom_precise_time
0,0x1FDD5BE63E904A3230A0AE9AA0E40C61,28,7520,54,2023-09-29,08:59:40
1,0x1FFC5D30D96AE2C4524842A09AA67C44,84,5085,24,2023-09-29,11:08:22
2,0x65F9DAD54040FBE48972B10F3B6337DC,25,6718,4,2023-09-29,08:51:28
3,0x747A26B4727A9A3094A1881F29D01510,208,6655,1,2023-09-29,22:32:48
4,0x81A8FFB4112FDBA4782A4FB50E1C0EE0,204,4472,46,2023-09-29,19:50:37
...,...,...,...,...,...,...
377159,0x51A33F38105E35D006C03F0694252007,52,5054,22,2023-09-29,08:48:38
377160,0x5BD2D7FBE1BE24A24AFD4D3ECA27FD3F,1,4662,6,2023-09-29,10:33:45
377161,0x6D191CC173F9415C1148B434D2F9012E,203,4810,5,2023-09-29,14:57:10
377162,0x7896A07D162530E734F876D16301EFEB,73,5176,23,2023-09-29,15:08:49


In [ ]:
#1.查看刷卡数据中有没有逆向做的？首先需要把od数据处理为前后相依的
#2.然后根据站点前后的大小来识别有多少是逆向的。删除逆向的，只保留正向的。

In [3]:
#获取路网结构数据
#把每个车的刷卡数据按照时间排序，然后按照递增序列分组，
#然后把每组保留每个站点的第一个数据
#然后根据同样的时间间隔进行填充。
with open('./datas/road_line_station_structure_setting.json', 'r') as f:
    temp_road_line_station_structure_setting = json.load(f)
temp_road_line_station_structure_setting
# 从文件中读取字典
with open('./datas/id_2_station.json', 'r') as file:
    id_2_station = json.load(file)
    id_2_station = {int(key): value for key, value in id_2_station.items()}
# 将字典保存到文件中
with open('./datas/station_2_id.json', 'r') as file:
    station_2_id = json.load(file)

In [4]:
def get_station_id(line_id,order_id):
    try:
        return station_2_id[temp_road_line_station_structure_setting[str(line_id)]['stations'][str(order_id)]]
    except:
        return -1
day_0929_dataframe['station_id'] = day_0929_dataframe.apply(lambda x: get_station_id(x['line_identity'],x['station_identity']),axis=1)
can_use_user_data = day_0929_dataframe[day_0929_dataframe['station_id']!=-1].reset_index(drop=True)[['custom_precise_time','station_id']]
can_use_user_data['hour'] = can_use_user_data.apply(lambda x:int(x['custom_precise_time'].split(':')[0]),axis=1)
can_use_user_data['minute'] = can_use_user_data.apply(lambda x:int(x['custom_precise_time'].split(':')[1]),axis=1)
can_use_user_data['seconds'] = can_use_user_data.apply(lambda x:int(x['custom_precise_time'].split(':')[2]),axis=1)
can_use_user_data

/tmp/ipykernel_22541/3133529963.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_0929_dataframe['station_id'] = day_0929_dataframe.apply(lambda x: get_station_id(x['line_identity'],x['station_identity']),axis=1)


,custom_precise_time,station_id,hour,minute,seconds
0,08:59:40,1255,8,59,40
1,11:08:22,516,11,8,22
2,08:51:28,1140,8,51,28
3,22:32:48,1257,22,32,48
4,19:50:37,933,19,50,37
...,...,...,...,...,...
338281,08:48:38,759,8,48,38
338282,10:33:45,349,10,33,45
338283,14:57:10,249,14,57,10
338284,15:08:49,1058,15,8,49


In [5]:
#时间精度 minutes
time_accurate = 8
time_split_array = list(range(0,60+time_accurate,time_accurate))
labels = list(range(len(time_split_array)-1))
can_use_user_data['minute_group'] = pd.cut(can_use_user_data['minute'], bins=time_split_array, right=False, labels=labels)
different_time_user_produce = can_use_user_data.groupby(['station_id','hour','minute_group'],observed=False).size().reset_index(name='count')
different_time_user_produce

,station_id,hour,minute_group,count
0,0,5,0,0
1,0,5,1,0
2,0,5,2,0
3,0,5,3,0
4,0,5,4,0
...,...,...,...,...
177531,1448,23,3,0
177532,1448,23,4,0
177533,1448,23,5,0
177534,1448,23,6,0


In [6]:
#当前时刻
now_hour = 8
now_minute = 23
#更新全部站点的情况，dataframe为站点列表，时间，minute
id_2_station = {int(key): value for key, value in id_2_station.items()}
station_id_list = list(id_2_station.keys())
conditions_df=pd.DataFrame({'station_id':station_id_list,\
                            'hour':[now_hour]*len(station_id_list),\
                            'minute_group':[now_minute//time_accurate]*len(station_id_list)})
#使用merge进行合并更新
merged_df = conditions_df.merge(different_time_user_produce, on=['station_id', 'hour', 'minute_group'], how='left').fillna(0)
merged_df


/tmp/ipykernel_22541/729591327.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df = conditions_df.merge(different_time_user_produce, on=['station_id', 'hour', 'minute_group'], how='left').fillna(0)


,station_id,hour,minute_group,count
0,0,8,2,0.0
1,1,8,2,0.0
2,2,8,2,6.0
3,3,8,2,0.0
4,4,8,2,0.0
...,...,...,...,...
1444,1444,8,2,0.0
1445,1445,8,2,0.0
1446,1446,8,2,0.0
1447,1447,8,2,0.0


In [ ]:
#根据当前时刻各个站点的出发人数的dataframe确定各个路径的选择情况。
#由于不同的站点面临的路径数是不同的，所以这时候不能再使用dataframe了
#只能使用字典。
#需要做几件事
#1.确定每个站点的可用的路径，以及对应的概率分布。
#2.确定各个站点